In [1]:
from datasets import load_dataset

dataset = load_dataset('boapps/kmdb_base')

<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileInfo size changed, may indicate binary incompatibility. Expected 64 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileSelector size changed, may indicate binary incompatibility. Expected 48 from C header, got 72 from PyObject
Found cached dataset parquet (/home/boa/.cache/huggingface/datasets/boapps___parquet/boapps--kmdb_base-c84a1bff36ea1db3/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
import spacy
spacy.require_gpu()

True

In [3]:
import hu_core_news_trf

nlp = hu_core_news_trf.load()

In [4]:
def get_entities(articles):
    results = {'entities': [], 'paragraph': [], 'url': []}
    for url, text in zip(articles['url'], articles['text']):
        try:
            for doc in nlp.pipe(text.split('\n'), disable=["tagger", "parser", "lemma_smoother"], ):
                results['entities'].append([(ent.lemma_, ent.label_) for ent in doc.ents])
                results['paragraph'].append(str(doc))
                results['url'].append(url)
        except Exception:
            print('error')
            #results['entities'].append(None)
    return results

entity_dataset = dataset.map(get_entities, batched=True, batch_size=1, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/1710 [00:00<?, ? examples/s]

Map:   0%|          | 0/3397 [00:00<?, ? examples/s]

Map:   0%|          | 0/26534 [00:00<?, ? examples/s]

In [5]:
filtered_entity_dataset = entity_dataset.filter(lambda row: len(row['paragraph']) > 100 and len(row['entities']) > 2)

Filter:   0%|          | 0/21815 [00:00<?, ? examples/s]

Filter:   0%|          | 0/42131 [00:00<?, ? examples/s]

Filter:   0%|          | 0/336831 [00:00<?, ? examples/s]

In [6]:
filtered_entity_dataset

DatasetDict({
    test: Dataset({
        features: ['url', 'entities', 'paragraph'],
        num_rows: 7721
    })
    validation: Dataset({
        features: ['url', 'entities', 'paragraph'],
        num_rows: 15126
    })
    train: Dataset({
        features: ['url', 'entities', 'paragraph'],
        num_rows: 120323
    })
})

In [7]:
filtered_entity_dataset.save_to_disk('../datasets/intermediate/re_dataset_url')

Flattening the indices:   0%|          | 0/7721 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/7721 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/15126 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/15126 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/120323 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/120323 [00:00<?, ? examples/s]